In [ ]:
import pandas as pd
import plotly.express as px

func_file = r"D:\sai\calling_function\most_common_calling_function_per_block.txt"
total_file = r"D:\sai\transactions_per_block.txt"

df_func = pd.read_csv(func_file, sep="\t")
df_total = pd.read_csv(total_file, sep="\t")
df_total = df_total.rename(columns={"count": "total_count"})

df = pd.merge(df_func, df_total, on="blockNumber")
df["percentage"] = (df["count"] / df["total_count"]) * 100
df["callingFunction"] = df["callingFunction"].astype(str)

df = df.sort_values("blockNumber")

df["rolling_percentage"] = df["percentage"].rolling(window=100, min_periods=1).mean()

fig = px.line(
    df,
    x="blockNumber",
    y="rolling_percentage",
    log_y=True,
    title="Smoothed Log-Scaled Transaction Percentage per Block",
    labels={"blockNumber": "Block Number", "rolling_percentage": "Smoothed % (Log Scale)"},
)

fig.update_traces(line=dict(width=1.5), opacity=0.7)
fig.update_layout(
    template="plotly_white",
    hovermode="x unified",
    dragmode="zoom",
    xaxis_title="Block Number",
    yaxis_title="Smoothed % (Log Scale)",  
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# File paths
block_file = r"D:\sai\calling_function\most_common_calling_function_per_block.txt"
global_file = r"D:\sai\calling_function\calling_function_counts.txt"
output_file = r"D:\sai\calling_function\block_function_usage_percent.txt"

# Load data
df_block = pd.read_csv(block_file, sep="\t")
df_global = pd.read_csv(global_file, sep="\t").rename(columns={"count": "total_count"})

# Convert datatypes
df_block["blockNumber"] = pd.to_numeric(df_block["blockNumber"], errors="coerce")
df_block["count"] = pd.to_numeric(df_block["count"], errors="coerce")
df_block["callingFunction"] = df_block["callingFunction"].astype(str)
df_global["total_count"] = pd.to_numeric(df_global["total_count"], errors="coerce")
df_global["callingFunction"] = df_global["callingFunction"].astype(str)

# Merge datasets
df = pd.merge(df_block, df_global, on="callingFunction", how="left")
df = df.dropna(subset=["total_count"])
df = df[df["total_count"] > 0]

# Filter out calling functions that appear in only one block
function_block_counts = df.groupby("callingFunction")["blockNumber"].nunique()
valid_functions = function_block_counts[function_block_counts > 1].index
df = df[df["callingFunction"].isin(valid_functions)]

# Calculate usage percentage per block
df["percentage"] = (df["count"] / df["total_count"]) * 100

# Get top-used function per block
df_top = df.sort_values(["blockNumber", "percentage"], ascending=[True, False])
df_top = df_top.groupby("blockNumber").first().reset_index()

# Save output
df_top[["blockNumber", "callingFunction", "count", "total_count", "percentage"]].to_csv(
    output_file, sep="\t", index=False
)

df_top["blockNumber"] = df_top["blockNumber"].astype(str)

fig = px.line(
    df_top,
    x="blockNumber",
    y="percentage",
    title="Top Calling Function Usage per Block (% of Global Total)",
    labels={"blockNumber": "Block Number", "percentage": "Usage % of Global"}
)

fig.update_traces(line=dict(color='blue'))
fig.update_layout(
    template="plotly_white",
    hovermode="x unified",
    dragmode="zoom",
    yaxis_title="Usage % (of Global)",
    xaxis_title="Block Number",
    xaxis=dict(type='category') 
)

fig.show()
